In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from array_io import *
from astropy import wcs


In [26]:
fname_img =  'F090W_8exp.fits'
hdulist = fits.open(fname_img)
header = hdulist[0].header
print header
nx = hdulist[0].header['NAXIS1']
ny = hdulist[0].header['NAXIS2']

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 2797                                                  NAXIS2  =                 2797                                                  EXTEND  =                    T                                                  COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H CTYPE1  = 'RA---TAN'                                                            CTYPE2  = 'DEC--TAN'                                                            CRVAL1  =           53.1153485                                                  CRPIX1  =                1399.                                                  CRVAL2  =          -27.8015545          

In [9]:
#convert pixel location of objectes extracted from image to ra and dec
def world(filename, x, y):
    hdu_list = fits.open(filename)
    w = wcs.WCS(hdu_list[0].header)
    hdu_list.close()
    wrd = w.all_pix2world(x, y, 0) 
    ra = wrd[:][0]
    dec = wrd[:][1]
    return ra, dec

In [20]:
#imports fits and puts into numpy array
def load_fits(filename):
    hdu_list = fits.open(filename, do_not_scale_image_data=True)
    tbdat = hdu_list[0].data
    tbdat = tbdat.byteswap().newbyteorder()
    return tbdat

In [10]:
m_AB_cat, ra_cat, dec_cat = read_three_arrays("25_bright_cat.txt")

In [13]:
m_AB_mosaic, ra_mosaic, dec_mosaic, x_mosaic, y_mosaic = read_five_arrays("25_bright_mosaic.txt")

In [15]:
ra, dec = world(fname_img, x_mosaic, y_mosaic)

In [16]:
print ra[0:10]

[ 53.11795846  53.10381878  53.10171086  53.10944839  53.11860113
  53.10642005  53.12469597  53.11105813  53.12502191  53.12013941]


In [17]:
print ra_mosaic[0:10]

[ 53.11795846  53.10381878  53.10171086  53.10944839  53.11860113
  53.10642005  53.12469597  53.11105813  53.12502191  53.12013941]


In [18]:
print ra_cat[0:10]

[ 53.118019  53.103878  53.101707  53.124931  53.10648   53.121548
  53.11866   53.124756  53.111118  53.125084]


In [21]:
tbdat = load_fits(fname_img)
tbdat = np.nan_to_num(tbdat)
for i in range(len(tbdat)):
    for j in range(len(tbdat)):
        if tbdat[i][j] == 0:
               tbdat[i][j] = 0.00001

In [22]:
#convert pixel location of corners of image to ra and dec
def world_area(filename, tbdat):
    hdu_list = fits.open(filename)
    w = wcs.WCS(hdu_list[0].header)
    hdu_list.close()
    world_min = w.all_pix2world(0,0,0)
    world_max = w.all_pix2world(len(tbdat), len(tbdat), 0)
    return world_min, world_max

In [23]:
world_min, world_max = world_area(fname_img, tbdat)
print world_min, world_max

[array(53.10148765369058), array(-27.813546904606454)] [array(53.12921619852619), array(-27.78955213252892)]


In [25]:
print ra_mosaic.max(), dec_mosaic.max()
print ra_mosaic.min(), dec_mosaic.min()

53.1277855597 -27.7896864632
53.1017108602 -27.811773562


In [28]:
x_test = (0,0,nx-1,nx-1)
y_test = (0,ny-1,0,ny-1)
ra_test, dec_test = world(fname_img, x_test, y_test)
print ra_test, dec_test

[ 53.10148765  53.10187856  53.1288215   53.12920629] [-27.8135469  -27.7892198  -27.81388789 -27.78956071]
